#**Ingesta a través de API rest**

#**Configuración de AWS CLI en Colab**


In [2]:
##Instalación de boto3 & request para la API de los datos
!pip install boto3
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 3.8 MB/s eta 0:00:00


In [3]:
import requests
import boto3
import json
from io import StringIO
import pandas as pd

In [ ]:
##Parametros de conexión con AWS###
!pip install awscli
!pip install --upgrade awscli

aws_access_key_id='____________________'
aws_secret_access_key='________________________________________'
aws_session_token='_____________________________________________________________'

In [ ]:
##Parametros del bucket##
s3_bucket='rawzonetrabajo1'
s3_target_file_name='raw/land_cov.csv'

#**Obteniendo la información del API que tiene el IMF**

####**Annual Surface Temperature Change**

In [18]:
##Consumiendo la API
##Esta se encuentra en un formato GeoJSON
url_land_cov = "https://services9.arcgis.com/weJ1QsnbMYJlCHdG/arcgis/rest/services/Indicator_3_5/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json"

#Realizamos el request
response_land_cov = requests.get(url_land_cov)
data_land_cov= response_land_cov.json()

##Los archivos GeoJSON son tipo de estructura de datos que son frecuentemente utilizados para datos geográficos
##Estos consisten en varios elementos principales, con roles especificos.
##Para este caso estamos usando el elemento de "features" y "properties". El primero es un array utilizado para almacenar propiedades de los objetos
##El segundo contiene data no geométrica sobre las propiedades, aquí se encuentra la metadata
features_land_cov = data_land_cov['features']

##Normalizamos el JSON
df_land_cov= pd.json_normalize(features_land_cov, record_path=None, meta=['id', 'geometry'], errors='ignore')


##Lo almacenamos en un DataFrame de Pandas
df_land_cov = pd.DataFrame([feature.get('attributes') for feature in features_land_cov])

In [23]:
##Vamos a convertir el DataFrame a CSV en memoria para el posterior almacenamiento en S3
csv_buffer = StringIO()
df_land_cov.to_csv(csv_buffer)

In [24]:
df_land_cov.to_csv('land_cov.csv', index=False)

In [ ]:
##Enviamos los datos a S3
s3.put_object(Body=csv_buffer.getvalue(),Bucket=s3_bucket,Key=s3_target_file_name)

NameError: name 's3' is not defined